# **Science Fair Project**
**Project Goal:**

To create a novel deep learning architecture method that could improve upon previous methods, using a method similar to boosting in machine learning.

**Basic Explanation of Model:**

A first deep learning model is trained on the original data. The training data that the first model misclassifies is then used to train a second model. That second model is then finetuned on the whole dataset. The training data that the second model misclassifies is then used to train a third model. That third model is then also finetuned on the whole dataset. The training data that the third model misclassifies is then used to finetune the first model.

After this, the predictions from all 3 models are then used as training data for a final Artificial Neural Network which makes combined predictions. Pictured below is a visual representation of the process.

![alt text](https://lh6.googleusercontent.com/QRKAMWUAlJ__bRkQeVygT5IjJQ4n8H9aXA9o0SIk0t7FjoiygbWRR9Ar5hq0I_8tPd9GIdZMVenbvAYX0pQHe9dFC6ilo2GHVph1VEFa)

**Challenges:**

The proposed architecture had to go through many iterations. The first model didn't contain any finetuning, and only worked on certain datasets. The second model had finetuning for both the 2nd and 3rd models, but not for the first, and did not work as intended. Finally, the third iteration of the model featured the original model finetuning, allowing the model to go full circle and have each model improve on eachother's weaknesses.

Also, there were many challenges not related to the results of the finished product. The procedure was applied on traditional machine learning applications (i.e. those in csv format), but did not produce any significant results despite significant experimentation with structure and hyperparameters, and it actually made performance decrease.

Also, there were other challenges related to portions of the project that were eventually scrapped. The project started as a driver safety project, with the intent to build an accident hotspot app. Said app was built, but the project continued to expand to detecting horizontal gaze nystagmus in apps, and then classifying distracted driver behavior from images. The distracted driver image classification ended up being the strongest portion of the project, and we scrapped the previous portions to focus on the innovative architecture we developed.

On top of all these problems, the project itself was a massive challenge, as when we started this project a little under than a year ago, neither of us knew how to code. We spent many sleepless nights working and persevering through complications and learned valuable skills in machine learning.

**Reception**

For our project, we were able to achieve 2nd Place in the category of Robotics and Intelligent Machines at the Texas Science and Engineering Fair (TXSEF).

We also summarized the results of our work in a professional-style paper and submitted it to multiple competitions to share our concept with the machine learning community to validate and build upon.



# **Section 1 - Image Classification**

This section tested the architecture against the popular CiFAR-10 Dataset, with 10 classes

This section was done by my partner, so excuse it if it is a little jumbled, because I wasn't quite sure how to order it. Since I didn't want to delete any critical part of the process, I simply copy and pasted the original code. In terms of understanding how the model works, I would recommend looking at the latter two sections (my sections), as they are clearer, edited to be more succint, and better demonstrate the goal. This first section did achieve more significant results, due to the difference in data tested (Image vs. database data; Deep Learning Models vs ANN's).

In [0]:
checkpoint = ModelCheckpoint('cnn1_cifar10.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model1_history = model1.fit_generator(datagen.flow(x_train, y_train_cat, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=20,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint])

In [0]:
import keras
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
from keras.callbacks import ModelCheckpoint

def lr_schedule(epoch):
  lrate = 0.001
  if epoch > 75:
    lrate = 0.0005
  if epoch > 100:
    lrate = 0.0003
  return lrate
 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train_cat = np_utils.to_categorical(y_train,num_classes)
y_test_cat = np_utils.to_categorical(y_test,num_classes)

In [0]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)
 
#training
batch_size = 64

In [0]:
from keras.models import load_model
model1 = load_model('cnn1_cifar10.h5')
model2 = load_model('cnn2_cifar10_finetuned.h5')
model3 = load_model('cnn3_cifar10_finetuned_20epochs.h5')
model4 = load_model('cnn4_cifar10_finetuned.h5')

In [0]:
# asses model performance & RETURN predicted values
from sklearn.metrics import classification_report, confusion_matrix
def performance(model, compare_imgs, right_classes):
  predictions_train = model.predict(compare_imgs)
  rounded_real_train = []
  for item in predictions_train:
    maxx = max(item)
    rounded_real_train.append(np.where(item==maxx)[0][0])

  print(classification_report(right_classes, rounded_real_train))  
  print(confusion_matrix(right_classes,rounded_real_train))
  return rounded_real_train

In [0]:
model1_wrong_indexes,model1_wrong_imgs,model1_wrong_img_class  = generate_error_imgs2(model1_revist_pred_train,x_train,y_train)
model1_wrong_imgs = np.array(model1_wrong_imgs)

model1_wrong_img_class = np.array(model1_wrong_img_class)
model1_wrong_img_class_cat = np_utils.to_categorical(model1_wrong_img_class,num_classes)

In [0]:
array = model1_pred != y_test_new

In [0]:
model1_pred = performance(model1,x_test,y_test)
model2_pred = performance(model2,x_test,y_test)
model3_pred = performance(model3,x_test,y_test)
model4_pred = performance(model4,x_test,y_test)

In [0]:
def progress_count(predictions1, predictions2, predictions3, predictions4, true_values):
  first_wrong = (predictions1 != true_values)
  second_wrong = (predictions2 != true_values)
  third_wrong = (predictions3 != true_values)
  fourth_wrong = (predictions4 != true_values)
  second_and_third_wrong = third_wrong.copy()
  num_second_third_overwrong=0
  num_first_second_fourth_overwrong=0
  num_first_third_fourth_overwrong=0
  num_second_third_fourth_overwrong=0
  num_first_second_third_fourth_overwrong=0
  
  num_first_wrong = sum(first_wrong)
  num_first_second_wrong = 0
  num_first_third_wrong = 0
  
  num_second_third_wrong = 0
  
  num_first_fourth_wrong = 0
  num_second_fourth_wrong = 0
  num_third_fourth_wrong = 0
  num_NET_third_first_wrong = 0
  
  index0 = -1
  for result in second_wrong:
    index0 +=1
    
    if result == 1 and result == first_wrong[index0]:
      num_first_second_wrong += 1
    if result == 1 and third_wrong[index0]==0:
      second_and_third_wrong[index0]=1
  num_second_else_wrong = sum(second_wrong)-num_first_second_wrong

  index1 = -1
  for result1 in third_wrong:
    index1 += 1
    if result1 == 1 and result1 == first_wrong[index1]:
      num_first_third_wrong += 1
    if result1 == 1 and result1== second_wrong[index1]:
      num_second_third_wrong += 1
    if result1 == 1 and result1== second_wrong[index1] and result1 == first_wrong[index1]:
      num_second_third_overwrong += 1
  num_third_else_wrong = sum(third_wrong)-num_first_third_wrong-num_second_third_wrong+num_second_third_overwrong

  index2 = -1
  for result2 in second_and_third_wrong:
    index2+=1
    if result2 == 1 and result2 == first_wrong[index2]:
      num_NET_third_first_wrong+= 1
  
  index3 = -1
  for result3 in fourth_wrong:
    index3 += 1
    if result3 == 1 and result3 == first_wrong[index3]:
      num_first_fourth_wrong += 1
    if result3 == 1 and result3 == second_wrong[index3]:
      num_second_fourth_wrong += 1
    if result3 == 1 and result3== second_wrong[index3] and result3 == first_wrong[index3]:
      num_first_second_fourth_overwrong += 1
    if result3 == 1 and result3 == third_wrong[index3]:
      num_third_fourth_wrong += 1
    if result3 == 1 and result3== third_wrong[index3] and result3 == second_wrong[index3]:
      num_second_third_fourth_overwrong += 1  
    if result3 == 1 and result3== third_wrong[index3] and result3 == first_wrong[index3]:
      num_first_third_fourth_overwrong += 1
    if result3 == 1 and result3== third_wrong[index3] and result3 == first_wrong[index3] and result3 == second_wrong[index3]:
      num_first_second_third_fourth_overwrong += 1
    
  num_fourth_else_wrong = sum(fourth_wrong)-num_first_fourth_wrong-num_second_fourth_wrong+num_first_second_fourth_overwrong-num_third_fourth_wrong+num_second_third_fourth_overwrong+num_first_third_fourth_overwrong+num_first_second_third_fourth_overwrong
  
  return(num_first_wrong,num_first_second_wrong,num_second_else_wrong,num_first_third_wrong,num_second_third_wrong,num_third_else_wrong,num_NET_third_first_wrong,num_first_fourth_wrong,num_second_fourth_wrong,num_third_fourth_wrong,num_fourth_else_wrong)

In [0]:
y_test_truth=y_test.reshape(10000)

In [0]:
a,b,c,d,e,f,g,h,i,j,k=progress_count(model1_pred,model2_pred,model3_pred,model4_pred,y_test_truth)
print(a,b,c,d,e,f,g,h,i,j,k)

In [0]:
import matplotlib.pyplot as plt

def plot_training_history(results):
    # Get the classification accuracy and loss-value
    # for the training-set.
    acc = results.history['acc']
    loss = results.history['loss']

    # Get it for the validation-set (we only use the test-set).
    try:
      val_acc = results.history['val_acc']
      val_loss = results.history['val_loss']
    except:
      pass
    # Plot the accuracy and loss-values for the training-set.
    plt.plot(acc, linestyle='-', color='b', label='Training Acc.')
    plt.plot(loss, 'o', color='b', label='Training Loss')
    
    # Plot it for the test-set.
    try:
      plt.plot(val_acc, linestyle='--', color='r', label='Test Acc.')
      plt.plot(val_loss, 'o', color='r', label='Test Loss')
    except:
      pass
    # Plot title and legend.
    plt.title('Training and Test Accuracy')
    plt.legend()

    # Ensure the plot shows correctly.
    plt.show()

In [0]:
def generate_error_imgs2(predictions, orig_imgs, real_values):
  index = -1
  count = 0
  wrong_img_indexes = []
  wrong_img_list = []
  wrong_img_class_list = []
  for item in predictions:
    index+=1
    if item != real_values[index]:
      count +=1
      wrong_img_indexes.append(index)
  print(count)
  for index2 in wrong_img_indexes:
    wrong_img_list.append(orig_imgs[index2])
    wrong_img_class_list.append(real_values[index2][0])
      
  return wrong_img_indexes, wrong_img_list,wrong_img_class_list

In [0]:
from sklearn.utils.class_weight import compute_class_weight
class_weight2 = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(model1_wrong_img_class),
                                    y=model1_wrong_img_class)
class_weight2=class_weight2*class_weight2
class_weight2

In [0]:
checkpoint2 = ModelCheckpoint('cnn2_cifar10_finetuned.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

'''model2_history = model2.fit_generator(datagen.flow(model1_wrong_imgs, model1_wrong_img_class_cat, batch_size=32),\
                    steps_per_epoch=x_train.shape[0] // 32,epochs=30,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint2],class_weight = class_weight2)'''
model2_finetune_history = model2.fit_generator(datagen.flow(x_train, y_train_cat, batch_size=64),\
                    steps_per_epoch=x_train.shape[0] // 64,epochs=20,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint2])
plot_training_history(model2_finetune_history)

In [0]:
model2=load_model('cnn2_cifar10_finetuned.h5')

In [0]:
model2_pred_train = performance(model2,x_train,y_train)

In [0]:
model2_pred_test = performance(model2,x_test,y_test)

In [0]:
model2_pred_wrongimgs = performance(model2,model1_wrong_imgs,model1_wrong_img_class)

In [0]:
model1_pred_wrongimgs = performance(model1,model1_wrong_imgs,model1_wrong_img_class)

In [0]:
model1_pred_test = performance(model1,x_test,y_test)

In [0]:
model2_wrong_indexes,model2_wrong_imgs,model2_wrong_img_class  = generate_error_imgs2(model2_pred_train,x_train,y_train)
model2_wrong_imgs = np.array(model2_wrong_imgs)

model2_wrong_img_class = np.array(model2_wrong_img_class)
model2_wrong_img_class_cat = np_utils.to_categorical(model2_wrong_img_class,num_classes)

In [0]:
from sklearn.utils.class_weight import compute_class_weight
class_weight3 = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(model2_wrong_img_class),
                                    y=model2_wrong_img_class)
class_weight3=class_weight3*class_weight3
class_weight3

In [0]:
from sklearn.utils.class_weight import compute_class_weight
class_weight3 = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(res2_wrong_img_class),
                                    y=res2_wrong_img_class)
class_weight3=class_weight3*class_weight3
class_weight3

In [0]:
plot_training_history(resnet3_history)

In [0]:
checkpoint3 = ModelCheckpoint('cnn3_cifar10_finetuned.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

'''model3_history = model3.fit_generator(datagen.flow(model2_wrong_imgs, model2_wrong_img_class_cat, batch_size=32),\
                    steps_per_epoch=x_train.shape[0] // 32,epochs=30,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint3],class_weight = class_weight3)'''
model3_finetune_history = model3.fit_generator(datagen.flow(x_train, y_train_cat, batch_size=64),\
                    steps_per_epoch=x_train.shape[0] // 64,epochs=10,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint3])
plot_training_history(model3_finetune_history)

In [0]:
plot_training_history(model3_finetune_history)

In [0]:
model3_pred_wrongimgs = performance(model3,model2_wrong_imgs, model2_wrong_img_class)

In [0]:
model3_pred_test = performance(model3,x_test, y_test)

In [0]:
model3_pred_train = performance(model3,x_train, y_train)

In [0]:
model3_wrong_indexes,model3_wrong_imgs,model3_wrong_img_class  = generate_error_imgs2(model3_pred_train,x_train,y_train)
model3_wrong_imgs = np.array(model3_wrong_imgs)

model3_wrong_img_class = np.array(model3_wrong_img_class)
model3_wrong_img_class_cat = np_utils.to_categorical(model3_wrong_img_class,num_classes)

In [0]:
from sklearn.utils.class_weight import compute_class_weight
class_weight4 = compute_class_weight(class_weight='balanced',
                                    classes=np.unique(model3_wrong_img_class),
                                    y=model3_wrong_img_class)
class_weight4=class_weight4*class_weight4
class_weight4

In [0]:
checkpoint1_pt2 = ModelCheckpoint('cnn1_cifar10_refinetuned.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

'''model1_history = model1.fit_generator(datagen.flow(model3_wrong_imgs, model3_wrong_img_class_cat, batch_size=32),\
                    steps_per_epoch=x_train.shape[0] // 32,epochs=20,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint1_pt2],class_weight = class_weight4)'''
model1_refinetune_history = model1.fit_generator(datagen.flow(x_train, y_train_cat, batch_size=64),\
                    steps_per_epoch=x_train.shape[0] // 64,epochs=10,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint1_pt2])
plot_training_history(model1_refinetune_history)

In [0]:
model1_refinetuned_pred_wrong1 = performance(model1,model1_wrong_imgs,model1_wrong_img_class)

In [0]:
model1_refinetuned_pred_wrong3 = performance(model1,model3_wrong_imgs,model3_wrong_img_class)

In [0]:
model1_pred_test = performance(model1,x_test,y_test)

In [0]:
#model1_pred_vals = model1.predict(x_train)
#model1_pred_vals = model2.predict(x_train)
#model3_pred_vals = model3.predict(x_train)
model4_pred_vals = model4.predict(x_train)

In [0]:
'''model1_pred_test_vals = model1.predict(x_test)
model2_pred_test_vals = model2.predict(x_test)
model3_pred_test_vals = model3.predict(x_test)'''
model4_pred_test_vals = model4.predict(x_test)

In [0]:
model1_pred_vals

In [0]:
model1_pred_vals = performance(model1,x_train,y_train)

In [0]:
model1_pred_vals_0 = [i[0] for i in model1_pred_vals]
model1_pred_vals_1 = [i[1] for i in model1_pred_vals]
model1_pred_vals_2 = [i[2] for i in model1_pred_vals]
model1_pred_vals_3 = [i[3] for i in model1_pred_vals]
model1_pred_vals_4 = [i[4] for i in model1_pred_vals]
model1_pred_vals_5 = [i[5] for i in model1_pred_vals]
model1_pred_vals_6 = [i[6] for i in model1_pred_vals]
model1_pred_vals_7 = [i[7] for i in model1_pred_vals]
model1_pred_vals_8 = [i[8] for i in model1_pred_vals]
model1_pred_vals_9 = [i[9] for i in model1_pred_vals]

reshaped_model2_pred_vals_0 = [i[0] for i in model2_pred_vals]
reshaped_model2_pred_vals_1 = [i[1] for i in model2_pred_vals]
reshaped_model2_pred_vals_2 = [i[2] for i in model2_pred_vals]
reshaped_model2_pred_vals_3 = [i[3] for i in model2_pred_vals]
reshaped_model2_pred_vals_4 = [i[4] for i in model2_pred_vals]
reshaped_model2_pred_vals_5 = [i[5] for i in model2_pred_vals]
reshaped_model2_pred_vals_6 = [i[6] for i in model2_pred_vals]
reshaped_model2_pred_vals_7 = [i[7] for i in model2_pred_vals]
reshaped_model2_pred_vals_8 = [i[8] for i in model2_pred_vals]
reshaped_model2_pred_vals_9 = [i[9] for i in model2_pred_vals]

reshaped_model3_pred_vals_0 = [i[0] for i in model3_pred_vals]
reshaped_model3_pred_vals_1 = [i[1] for i in model3_pred_vals]
reshaped_model3_pred_vals_2 = [i[2] for i in model3_pred_vals]
reshaped_model3_pred_vals_3 = [i[3] for i in model3_pred_vals]
reshaped_model3_pred_vals_4 = [i[4] for i in model3_pred_vals]
reshaped_model3_pred_vals_5 = [i[5] for i in model3_pred_vals]
reshaped_model3_pred_vals_6 = [i[6] for i in model3_pred_vals]
reshaped_model3_pred_vals_7 = [i[7] for i in model3_pred_vals]
reshaped_model3_pred_vals_8 = [i[8] for i in model3_pred_vals]
reshaped_model3_pred_vals_9 = [i[9] for i in model3_pred_vals]

reshaped_model4_pred_vals_0 = [i[0] for i in model4_pred_vals]
reshaped_model4_pred_vals_1 = [i[1] for i in model4_pred_vals]
reshaped_model4_pred_vals_2 = [i[2] for i in model4_pred_vals]
reshaped_model4_pred_vals_3 = [i[3] for i in model4_pred_vals]
reshaped_model4_pred_vals_4 = [i[4] for i in model4_pred_vals]
reshaped_model4_pred_vals_5 = [i[5] for i in model4_pred_vals]
reshaped_model4_pred_vals_6 = [i[6] for i in model4_pred_vals]
reshaped_model4_pred_vals_7 = [i[7] for i in model4_pred_vals]
reshaped_model4_pred_vals_8 = [i[8] for i in model4_pred_vals]
reshaped_model4_pred_vals_9 = [i[9] for i in model4_pred_vals]

In [0]:

model1_pred_train_vals2_0 = [i[0] for i in model1_pred_train_vals2]
model1_pred_train_vals2_1 = [i[1] for i in model1_pred_train_vals2]
model1_pred_train_vals2_2 = [i[2] for i in model1_pred_train_vals2]
model1_pred_train_vals2_3 = [i[3] for i in model1_pred_train_vals2]
model1_pred_train_vals2_4 = [i[4] for i in model1_pred_train_vals2]
model1_pred_train_vals2_5 = [i[5] for i in model1_pred_train_vals2]
model1_pred_train_vals2_6 = [i[6] for i in model1_pred_train_vals2]
model1_pred_train_vals2_7 = [i[7] for i in model1_pred_train_vals2]
model1_pred_train_vals2_8 = [i[8] for i in model1_pred_train_vals2]
model1_pred_train_vals2_9 = [i[9] for i in model1_pred_train_vals2]

In [0]:
model1_pred_test_vals2_0 = [i[0] for i in model1_pred_test_vals2]
model1_pred_test_vals2_1 = [i[1] for i in model1_pred_test_vals2]
model1_pred_test_vals2_2 = [i[2] for i in model1_pred_test_vals2]
model1_pred_test_vals2_3 = [i[3] for i in model1_pred_test_vals2]
model1_pred_test_vals2_4 = [i[4] for i in model1_pred_test_vals2]
model1_pred_test_vals2_5 = [i[5] for i in model1_pred_test_vals2]
model1_pred_test_vals2_6 = [i[6] for i in model1_pred_test_vals2]
model1_pred_test_vals2_7 = [i[7] for i in model1_pred_test_vals2]
model1_pred_test_vals2_8 = [i[8] for i in model1_pred_test_vals2]
model1_pred_test_vals2_9 = [i[9] for i in model1_pred_test_vals2]

In [0]:
model1_pred_test_vals_0 = [i[0] for i in model1_pred_test_vals]
model1_pred_test_vals_1 = [i[1] for i in model1_pred_test_vals]
model1_pred_test_vals_2 = [i[2] for i in model1_pred_test_vals]
model1_pred_test_vals_3 = [i[3] for i in model1_pred_test_vals]
model1_pred_test_vals_4 = [i[4] for i in model1_pred_test_vals]
model1_pred_test_vals_5 = [i[5] for i in model1_pred_test_vals]
model1_pred_test_vals_6 = [i[6] for i in model1_pred_test_vals]
model1_pred_test_vals_7 = [i[7] for i in model1_pred_test_vals]
model1_pred_test_vals_8 = [i[8] for i in model1_pred_test_vals]
model1_pred_test_vals_9 = [i[9] for i in model1_pred_test_vals]

reshaped_model2_pred_test_vals_0 = [i[0] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_1 = [i[1] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_2 = [i[2] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_3 = [i[3] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_4 = [i[4] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_5 = [i[5] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_6 = [i[6] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_7 = [i[7] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_8 = [i[8] for i in model2_pred_test_vals]
reshaped_model2_pred_test_vals_9 = [i[9] for i in model2_pred_test_vals]

reshaped_model3_pred_test_vals_0 = [i[0] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_1 = [i[1] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_2 = [i[2] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_3 = [i[3] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_4 = [i[4] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_5 = [i[5] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_6 = [i[6] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_7 = [i[7] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_8 = [i[8] for i in model3_pred_test_vals]
reshaped_model3_pred_test_vals_9 = [i[9] for i in model3_pred_test_vals]

reshaped_model4_pred_test_vals_0 = [i[0] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_1 = [i[1] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_2 = [i[2] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_3 = [i[3] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_4 = [i[4] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_5 = [i[5] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_6 = [i[6] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_7 = [i[7] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_8 = [i[8] for i in model4_pred_test_vals]
reshaped_model4_pred_test_vals_9 = [i[9] for i in model4_pred_test_vals]

In [0]:
# Test ANN
import pandas as pd
dataset4 = pd.DataFrame({'first0': model1_pred_test_vals_0,
                        'first1': model1_pred_test_vals_1,
                        'first2': model1_pred_test_vals_2,
                        'first3': model1_pred_test_vals_3,
                        'first4': model1_pred_test_vals_4,
                        'first5': model1_pred_test_vals_5,
                        'first6': model1_pred_test_vals_6,
                        'first7': model1_pred_test_vals_7,
                        'first8': model1_pred_test_vals_8,
                        'first9': model1_pred_test_vals_9,
                        'inverse0': reshaped_model2_pred_test_vals_0,
                        'inverse1': reshaped_model2_pred_test_vals_1,
                        'inverse2': reshaped_model2_pred_test_vals_2,
                        'inverse3': reshaped_model2_pred_test_vals_3,
                        'inverse4': reshaped_model2_pred_test_vals_4,
                        'inverse5': reshaped_model2_pred_test_vals_5,
                        'inverse6': reshaped_model2_pred_test_vals_6,
                        'inverse7': reshaped_model2_pred_test_vals_7,
                        'inverse8': reshaped_model2_pred_test_vals_8,
                        'inverse9': reshaped_model2_pred_test_vals_9,
                        'invverse0': reshaped_model3_pred_test_vals_0,
                        'invverse1': reshaped_model3_pred_test_vals_1,
                        'invverse2': reshaped_model3_pred_test_vals_2,
                        'invverse3': reshaped_model3_pred_test_vals_3,
                        'invverse4': reshaped_model3_pred_test_vals_4,
                        'invverse5': reshaped_model3_pred_test_vals_5,
                        'invverse6': reshaped_model3_pred_test_vals_6,
                        'invverse7': reshaped_model3_pred_test_vals_7,
                        'invverse8': reshaped_model3_pred_test_vals_8,
                        'invverse9': reshaped_model3_pred_test_vals_9,
                        'invvverse0': reshaped_model4_pred_test_vals_0,
                        'invvverse1': reshaped_model4_pred_test_vals_1,
                        'invvverse2': reshaped_model4_pred_test_vals_2,
                        'invvverse3': reshaped_model4_pred_test_vals_3,
                        'invvverse4': reshaped_model4_pred_test_vals_4,
                        'invvverse5': reshaped_model4_pred_test_vals_5,
                        'invvverse6': reshaped_model4_pred_test_vals_6,
                        'invvverse7': reshaped_model4_pred_test_vals_7,
                        'invvverse8': reshaped_model4_pred_test_vals_8,
                        'invvverse9': reshaped_model4_pred_test_vals_9,
                        'correct':y_test_reshape}, 
                       index=np.arange(len(y_test_reshape)),columns=['first0',
                                                                   'first1',
                                                                   'first2',
                                                                   'first3',
                                                                   'first4',
                                                                   'first5',
                                                                   'first6',
                                                                   'first7',
                                                                   'first8',
                                                                   'first9',
                                                                   'inverse0',
                                                                   'inverse1',
                                                                   'inverse2',
                                                                   'inverse3',
                                                                   'inverse4',
                                                                   'inverse5',
                                                                   'inverse6',
                                                                   'inverse7',
                                                                   'inverse8',
                                                                   'inverse9',
                                                                   'invverse0',
                                                                   'invverse1',
                                                                   'invverse2',
                                                                   'invverse3',
                                                                   'invverse4',
                                                                   'invverse5',
                                                                   'invverse6',
                                                                   'invverse7',
                                                                   'invverse8',
                                                                   'invverse9',
                                                                   'invvverse0',
                                                                   'invvverse1',
                                                                   'invvverse2',
                                                                   'invvverse3',
                                                                   'invvverse4',
                                                                   'invvverse5',
                                                                   'invvverse6',
                                                                   'invvverse7',
                                                                   'invvverse8',
                                                                   'invvverse9',
                                                                   'correct'])

In [0]:
y_train_reshape=y_train.reshape(len(y_train))

In [0]:
import pandas as pd
dataset2_train = pd.DataFrame({'first0': model1_pred_vals_0,
                        'first1': model1_pred_vals_1,
                        'first2': model1_pred_vals_2,
                        'first3': model1_pred_vals_3,
                        'first4': model1_pred_vals_4,
                        'first5': model1_pred_vals_5,
                        'first6': model1_pred_vals_6,
                        'first7': model1_pred_vals_7,
                        'first8': model1_pred_vals_8,
                        'first9': model1_pred_vals_9,
                        'inverse0': reshaped_model2_pred_vals_0,
                        'inverse1': reshaped_model2_pred_vals_1,
                        'inverse2': reshaped_model2_pred_vals_2,
                        'inverse3': reshaped_model2_pred_vals_3,
                        'inverse4': reshaped_model2_pred_vals_4,
                        'inverse5': reshaped_model2_pred_vals_5,
                        'inverse6': reshaped_model2_pred_vals_6,
                        'inverse7': reshaped_model2_pred_vals_7,
                        'inverse8': reshaped_model2_pred_vals_8,
                        'inverse9': reshaped_model2_pred_vals_9,
                        'invverse0': reshaped_model3_pred_vals_0,
                        'invverse1': reshaped_model3_pred_vals_1,
                        'invverse2': reshaped_model3_pred_vals_2,
                        'invverse3': reshaped_model3_pred_vals_3,
                        'invverse4': reshaped_model3_pred_vals_4,
                        'invverse5': reshaped_model3_pred_vals_5,
                        'invverse6': reshaped_model3_pred_vals_6,
                        'invverse7': reshaped_model3_pred_vals_7,
                        'invverse8': reshaped_model3_pred_vals_8,
                        'invverse9': reshaped_model3_pred_vals_9,
                        'invvverse0': reshaped_model4_pred_vals_0,
                        'invvverse1': reshaped_model4_pred_vals_1,
                        'invvverse2': reshaped_model4_pred_vals_2,
                        'invvverse3': reshaped_model4_pred_vals_3,
                        'invvverse4': reshaped_model4_pred_vals_4,
                        'invvverse5': reshaped_model4_pred_vals_5,
                        'invvverse6': reshaped_model4_pred_vals_6,
                        'invvverse7': reshaped_model4_pred_vals_7,
                        'invvverse8': reshaped_model4_pred_vals_8,
                        'invvverse9': reshaped_model4_pred_vals_9,
                        'correct':y_train_reshape}, 
                       index=np.arange(len(y_train_reshape)),columns=['first0',
                                                                   'first1',
                                                                   'first2',
                                                                   'first3',
                                                                   'first4',
                                                                   'first5',
                                                                   'first6',
                                                                   'first7',
                                                                   'first8',
                                                                   'first9',
                                                                   'inverse0',
                                                                   'inverse1',
                                                                   'inverse2',
                                                                   'inverse3',
                                                                   'inverse4',
                                                                   'inverse5',
                                                                   'inverse6',
                                                                   'inverse7',
                                                                   'inverse8',
                                                                   'inverse9',
                                                                   'invverse0',
                                                                   'invverse1',
                                                                   'invverse2',
                                                                   'invverse3',
                                                                   'invverse4',
                                                                   'invverse5',
                                                                   'invverse6',
                                                                   'invverse7',
                                                                   'invverse8',
                                                                   'invverse9',
                                                                   'invvverse0',
                                                                   'invvverse1',
                                                                   'invvverse2',
                                                                   'invvverse3',
                                                                   'invvverse4',
                                                                   'invvverse5',
                                                                   'invvverse6',
                                                                   'invvverse7',
                                                                   'invvverse8',
                                                                   'invvverse9',
                                                                   'correct'])

In [0]:
X_test_vals = dataset4.iloc[:, :40].values
y_test_vals = dataset4.iloc[:, 40].values

In [0]:
X_test_vals = dataset4.iloc[:, :30].values
y_test_vals = dataset4.iloc[:, 40].values

In [0]:
ann_predict3_test = ann_model3.predict(X_test_vals)
rounded_real_train = []
for item in ann_predict3_test:
  maxx = max(item)
  rounded_real_train.append(np.where(item==maxx)[0][0])

print(classification_report(y_test_vals, rounded_real_train))  
print(confusion_matrix(y_test_vals,rounded_real_train))

In [0]:
X3 = dataset2_train.iloc[:, :30].values
y3 = dataset2_train.iloc[:, 40].values

In [0]:
X = dataset2_train.iloc[:, :40].values
y = dataset2_train.iloc[:, 40].values

In [0]:
from sklearn.model_selection import train_test_split
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=91)

In [0]:
y_train3_cat = np_utils.to_categorical(y_train3,10)
y_test3_cat = np_utils.to_categorical(y_test3,10)

In [0]:
# Creates ann_model2- all training images
from keras.optimizers import Adam
weight_decay = 1e-4

ann_model3 = Sequential()

ann_model3.add(Dense(30, input_dim=30, activation='relu'))
ann_model3.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
ann_model3.add(Dropout(0.4))

ann_model3.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(weight_decay)))
ann_model3.add(Dropout(0.4))

# Last layer simple sigmoid function to output 0 or 1 (our label)
ann_model3.add(Dense(10, activation='softmax'))

ann_model3.compile(Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint_ann = ModelCheckpoint('cifar_ann_three_models_vals_refinetuned.h5', verbose=1, monitor='loss',save_best_only=True, mode='auto')  

ann_model3_results = ann_model3.fit(X_train3,y_train3_cat,epochs=20,  callbacks=[checkpoint_ann], verbose=1)

In [0]:
ann_model3=load_model('cifar_ann_three_models_vals_refinetuned.h5')

In [0]:
ann_predict_test = ann_model3.predict(X_test3)
rounded_real_train = []
for item in ann_predict_test:
  maxx = max(item)
  rounded_real_train.append(np.where(item==maxx)[0][0])

print(classification_report(y_test3, rounded_real_train))  
print(confusion_matrix(y_test3,rounded_real_train))

In [0]:
checkpoint4 = ModelCheckpoint('cnn4_cifar10_finetuned.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

'''model4_history = model4.fit_generator(datagen.flow(model3_wrong_imgs, model3_wrong_img_class_cat, batch_size=32),\
                    steps_per_epoch=x_train.shape[0] // 32,epochs=20,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint3],class_weight = class_weight4)'''
model4_finetune_history = model4.fit_generator(datagen.flow(x_train, y_train_cat, batch_size=64),\
                    steps_per_epoch=x_train.shape[0] // 64,epochs=10,\
                    verbose=1,validation_data=(x_test,y_test_cat),callbacks=[LearningRateScheduler(lr_schedule),checkpoint4])
plot_training_history(model4_finetune_history)

In [0]:
model4_pred_wrongimgs = performance(model4,model3_wrong_imgs, model3_wrong_img_class)

In [0]:
model4_pred_train = performance(model4,x_train, y_train)

In [0]:
model4_pred_test = performance(model4,x_test, y_test)

In [0]:
# data_gather: Test ann on firsttest

model1_predtest=np.array(model1_pred_test)
model2_predtest=np.array(model2_pred_test)
model3_predtest=np.array(model3_pred_test)
#model4_predtest=np.array(model4_pred_test)


model1_predtest_reshape = model1_predtest.reshape(len(model1_predtest))
model2_predtest_reshape = model2_predtest.reshape(len(model2_predtest))
model3_predtest_reshape = model3_predtest.reshape(len(model3_predtest))
#model4_predtest_reshape = model4_predtest.reshape(len(model4_predtest))

y_test_reshape = y_test.reshape(len(y_test))

dataset3_test = pd.DataFrame({'first': model1_predtest_reshape, 'inverse': model2_predtest_reshape,'third':model3_predtest_reshape,'correct':y_test_reshape}, index=np.arange(len(model3_predtest_reshape)),columns=['first', 'inverse','third','correct'])
#dataset_test4 = pd.DataFrame({'first': model1_predtest_reshape, 'second': model2_predtest_reshape, 'third':model3_predtest_reshape,'fourth':model4_predtest_reshape,'correct':y_test_reshape}, index=np.arange(len(model4_predtest_reshape)),columns=['first', 'second','third','fourth','correct'])


Meta3_X_test = dataset3_test.iloc[:,0:3]
Meta3_y_test = dataset3_test.iloc[:,3]

In [0]:
import pandas as pd
# data_gather: Train ann on both model's predictions on the original training data
model1_predtrain=np.array(model1_pred_vals)
model2_predtrain=np.array(model2_pred_vals)
model3_predtrain=np.array(model3_pred_train)
#model4_predtrain=np.array(model4_pred_train)


model1_predtrain_reshape = model1_predtrain.reshape(len(model1_predtrain))
model2_predtrain_reshape = model2_predtrain.reshape(len(model2_predtrain))
model3_predtrain_reshape = model3_predtrain.reshape(len(model3_predtrain))
#model4_predtrain_reshape = model4_predtrain.reshape(len(model4_predtrain))

y_train_reshape = y_train.reshape(len(y_train))

dataset = pd.DataFrame({'first': model1_predtrain_reshape, 'inverse': model2_predtrain_reshape, 'third':model3_predtrain_reshape,'correct':y_train_reshape}, index=np.arange(len(model1_predtrain_reshape)),columns=['first', 'inverse','third','correct'])
#dataset4 = pd.DataFrame({'first': model1_predtrain_reshape, 'inverse': model3_predtrain_reshape, 'third':model2_predtrain_reshape,'fourth':model4_predtrain_reshape,'correct':y_train_reshape}, index=np.arange(len(model1_predtrain_reshape)),columns=['first', 'inverse','third','fourth','correct'])

Meta3_X = dataset.iloc[:,0:3]
Meta3_y = dataset.iloc[:,3]

'''Meta2_X = dataset.iloc[:,0:2]
Meta2_y = dataset.iloc[:,3]

Meta4_X=dataset4.iloc[:,0:4]
Meta4_y=dataset4.iloc[:,4]'''


from sklearn.model_selection import train_test_split
Meta3_X_train,Meta3_X_test, Meta3_y_train, Meta3_y_test = train_test_split(Meta3_X, Meta3_y, test_size=0.3, random_state=91)
#Meta2_X_train,Meta2_X_test, Meta2_y_train, Meta2_y_test = train_test_split(Meta2_X, Meta2_y, test_size=0.3, random_state=91)
#Meta4_X_train,Meta4_X_test, Meta4_y_train, Meta4_y_test = train_test_split(Meta4_X, Meta4_y, test_size=0.3, random_state=91)

In [0]:
Meta3_y_train_cat = np_utils.to_categorical(Meta3_y_train,10)
Meta3_y_test_cat = np_utils.to_categorical(Meta3_y_test,10)

In [0]:
ann_predict_test = ann_model3.predict(Meta3_X_test)
rounded_real_train = []
for item in ann_predict_test:
  maxx = max(item)
  rounded_real_train.append(np.where(item==maxx)[0][0])

print(classification_report(Meta3_y_test, rounded_real_train))  
print(confusion_matrix(Meta3_y_test,rounded_real_train))

In [0]:
# Creates ann_model2- all training images
from keras.optimizers import Adam
ann_model3 = Sequential()

ann_model3.add(Dense(3, input_dim=3, activation='relu'))
ann_model3.add(Dense(256, activation='relu'))
ann_model3.add(Dropout(0.3))

ann_model3.add(Dense(256, activation='relu'))
ann_model3.add(Dropout(0.3))

# Last layer simple sigmoid function to output 0 or 1 (our label)
ann_model3.add(Dense(10, activation='softmax'))

ann_model3.compile(Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint_ann = ModelCheckpoint('cifar_ann_three_models.h5', verbose=1, monitor='loss',save_best_only=True, mode='auto')  

ann_model3_results = ann_model3.fit(Meta3_X_train,Meta3_y_train_cat,epochs=15,  callbacks=[checkpoint_ann], verbose=1)

In [0]:
ann_model3 = load_model('cifar_ann_three_models.h5')

In [0]:
# Creates ann_model2- all training images
from keras.optimizers import Adam
ann_model = Sequential()

ann_model.add(Dense(3, input_dim=3, activation='relu'))
ann_model.add(Dense(256, activation='relu'))
ann_model.add(Dropout(0.3))

ann_model.add(Dense(256, activation='relu'))
ann_model.add(Dropout(0.3))

# Last layer simple sigmoid function to output 0 or 1 (our label)
ann_model.add(Dense(10, activation='softmax'))

ann_model.compile(Adam(lr=0.01),loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint_ann = ModelCheckpoint('cifar_ann_three_models.h5', verbose=1, monitor='loss',save_best_only=True, mode='auto')  

ann_model_results = ann_model.fit(X_train,y_train_cat,epochs=15,  callbacks=[checkpoint_ann], verbose=1)

In [0]:
ann_predict3_test = ann_model3.predict(Meta3_X_test)
rounded_real_train = []
for item in ann_predict3_test:
  maxx = max(item)
  rounded_real_train.append(np.where(item==maxx)[0][0])

print(classification_report(Meta3_y_test, rounded_real_train))  
print(confusion_matrix(Meta3_y_test,rounded_real_train))

In [0]:
def predictions4(res1,res2,res3,res4):
  final_predictions = []
  for item in range(len(res1)):
    temp_list = []
    temp_list.append([res1[item],res2[item],res3[item],res4[item]])
    final_predictions.append(max(temp_list[0], key=temp_list[0].count))
  return final_predictions

In [0]:
meta_test_predictions = predictions4(model1_pred_test, model2_pred_test, model3_pred_test,model4_pred_test)

In [0]:
print(classification_report(y_test, meta_test_predictions))  
print(confusion_matrix(y_test,meta_test_predictions))

In [0]:
def predictions(res1,res2,res3):
  final_predictions = []
  for item in range(len(res1)):
    if res1[item] == res2[item]:
      final_predictions.append(res1[item])
    elif res1[item] == res3[item]:
      final_predictions.append(res1[item])
    elif res2[item] == res3[item]:
      final_predictions.append(res2[item])
    else:
      final_predictions.append(res2[item])
  return final_predictions

In [0]:
meta_test_predictions = predictions(model1_pred_test, model2_pred_test, model3_pred_test)

In [0]:
print(classification_report(y_test, meta_test_predictions))  
print(confusion_matrix(y_test,meta_test_predictions))

In [0]:
a,b,c,d,e,f,g=progress_count(model1_pred_test, model2_pred_test, model3_pred_test,y_test_truth)

In [0]:
print(a,b,c,d,e,f,g)

# Section 2 - Binary Machine Learning Classification

This section tested the architecture against a "Will it Rain?" binary dataset.

In [0]:
#read dataset
import pandas as pd
data = pd.read_csv('weatherAUS.csv')

In [0]:
#cut out irrelevant features
d1 = data.iloc[:,2:21]
d1.head()

In [0]:
#cut out irrelevant features
del d1['WindGustDir']
del d1['WindDir9am']
del d1['WindDir3pm']
d1.head()

In [0]:
#replace empty values
d1.fillna(0, inplace = True)
d1.head()

In [0]:
#get y-values and quantify values
dy = data.iloc[:,-1]
dy.replace(('Yes', 'No'), (1, 0), inplace=True)

In [0]:
#create training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(d1, dy, random_state=42)

In [0]:
#scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [0]:
#import packages
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np

In [0]:
#Create First Model
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 16))

classifier.add(Dropout(0.5))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train_scaled, y_train, batch_size = 10, epochs = 10)

In [0]:
#define incorrect image function
def generate_error_imgs2(predictions, orig_imgs, real_values):
  index = -1
  count = 0
  wrong_img_indexes = []
  wrong_img_list = []
  wrong_img_class_list = []
  for item in predictions:
    index+=1
    if item != real_values[index]:
      count +=1
      wrong_img_indexes.append(index)
  print(count)
  for index2 in wrong_img_indexes:
    wrong_img_list.append(orig_imgs[index2])
    wrong_img_class_list.append(real_values[index2])
      
  return wrong_img_indexes, wrong_img_list,wrong_img_class_list

In [0]:
#scale test data
X_test_scaled = scaler.transform(X_test)

In [0]:
#measure accuracy on test data
y_pred = classifier.predict(X_test_scaled)
y_pred = (y_pred > 0.5)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

In [0]:
#measure accuracy on train data
y_pred = classifier.predict(X_train_scaled)
y_pred = (y_pred > 0.5)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, y_pred))

In [0]:
#generate error images
wrong_index, wrong_x, wrong_y = generate_error_imgs2(y_pred,X_train_scaled, np.array(y_train))

In [0]:
#convert to arrays
wrong_x = np.array(wrong_x)
wrong_y = np.array(wrong_y)

In [0]:
#Create Second Model
# Initialising the ANN
classifier2 = Sequential()

# Adding the input layer and the first hidden layer
classifier2.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 16))

classifier2.add(Dropout(0.5))

# Adding the input layer and the first hidden layer
classifier2.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier2.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier2.fit(wrong_x, wrong_y, batch_size = 10, epochs = 10)

In [0]:
#Fine-tune on original data
classifier2.fit(X_train_scaled, y_train, batch_size = 10, epochs = 1)

In [0]:
#measure accuracy on test data
y_pred = classifier2.predict(X_test_scaled)
y_pred = (y_pred > 0.5)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

In [0]:
#measure accuracy on training data
y_pred = classifier2.predict(wrong_x)
y_pred = (y_pred > 0.5)

from sklearn.metrics import accuracy_score

print(accuracy_score(wrong_y, y_pred))

In [0]:
#generate error images
wrong_index2, wrong_x2, wrong_y2 = generate_error_imgs2(y_pred, wrong_x, wrong_y)

In [0]:
#convert to arrays
wrong_x2 = np.array(wrong_x2)
wrong_y2 = np.array(wrong_y2)

In [0]:
#Create Third Model
# Initialising the ANN
classifier3 = Sequential()

# Adding the input layer and the first hidden layer
classifier3.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 16))

classifier3.add(Dropout(0.5))

# Adding the input layer and the first hidden layer
classifier3.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier3.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier3.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier3.fit(wrong_x2, wrong_y2, batch_size = 10, epochs = 10)

In [0]:
#Fine-tune on original data
classifier3.fit(X_train_scaled, y_train, batch_size = 10, epochs = 1)

In [0]:
#measure accuracy on test data
y_pred = classifier3.predict(X_test_scaled)
y_pred = (y_pred > 0.5)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, y_pred))

In [0]:
#measure accuracy on training data
y_pred = classifier3.predict(wrong_x2)
y_pred = (y_pred > 0.5)

from sklearn.metrics import accuracy_score

print(accuracy_score(wrong_y2, y_pred))

In [0]:
#generate error images
wrong_index3, wrong_x3, wrong_y3 = generate_error_imgs2(y_pred, wrong_x2, wrong_y2)

In [0]:
#convert to arrays
wrong_x3 = np.array(wrong_x3)
wrong_y3 = np.array(wrong_y3)

In [0]:
#Fine-tune first model on last one's missed data
classifier.fit(wrong_x3, wrong_y3, batch_size = 10, epochs = 1)

In [0]:
#get final predictions for all
firstpred = classifier.predict(X_train_scaled)
secondpred = classifier2.predict(X_train_scaled)
thirdpred = classifier3.predict(X_train_scaled)

In [0]:
#Create master dataset
master = []
for i in range(len(X_train)):
  one = [None] * 3
  one[0] = firstpred[i][0]
  one[1] = secondpred[i][0]
  one[2] = thirdpred[i][0]
  master.append(one)

In [0]:
#create master arrays
master = np.array(master)
y_train = np.array(y_train)

In [0]:
#even out classes
zcount = 0
wcount = 0
indexes = []
for num in range(len(y_train)):
  if y_train[num] == 0:
    if zcount < 10000:
      indexes.append(num)
      zcount += 1
  elif y_train[num] == 1:
    if wcount < 10000:
      indexes.append(num)
      wcount += 1

In [0]:
#create even classes
fm = []
fy = []
for num in indexes:
  fm.append(master[num])
  fy.append(y_train[num])

In [0]:
#create final arrays
fm = np.array(fm)
fy = np.array(fy)

In [0]:
#train final classifier
# Initialising the ANN
mclassifier = Sequential()

# Adding the input layer and the first hidden layer
mclassifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = 3))

mclassifier.add(Dropout(0.5))

# Adding the second hidden layer
mclassifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
mclassifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
mclassifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
mclassifier.fit(fm, fy, batch_size = 10, epochs = 10)

In [0]:
#make test predictions
firstpred2 = classifier.predict(X_test)
secondpred2 = classifier2.predict(X_test)
thirdpred2 = classifier3.predict(X_test)

In [0]:
#create master test dataset
master2 = []
for i in range(len(X_test)):
  one2 = [None] * 3
  one2[0] = firstpred2[i][0]
  one2[1] = secondpred2[i][0]
  one2[2] = thirdpred2[i][0]
  master2.append(one2)
master2 = np.array(master2)

In [0]:
#get final accuracy
finalpred = mclassifier.predict(master2)
finalpred = (finalpred > 0.5)
from sklearn.metrics import confusion_matrix,classification_report
report = classification_report(y_test, finalpred)
print(report) 
print(confusion_matrix(y_test,finalpred))

# Section 3 - Multiclass Machine Learning Classification

This section tested the architecture against a multiclass Forest Covertype dataset.

In [0]:
#import dataset
from sklearn.datasets import fetch_covtype
cov_data = fetch_covtype()

In [0]:
#separate y-data into classes
cv1 = []
cv2 = []
cv3 = []
cv4 = []
cv5 = []
cv6 = []
cv7 = []
for num in range(len(cov_data.target)):
  if cov_data.target[num] == 1:
    cv1.append(num)
  elif cov_data.target[num] == 2:
    cv2.append(num)
  elif cov_data.target[num] == 3:
    cv3.append(num)
  elif cov_data.target[num] == 4:
    cv4.append(num)
  elif cov_data.target[num] == 5:
    cv5.append(num)
  elif cov_data.target[num] == 6:
    cv6.append(num)
  elif cov_data.target[num] == 7:
    cv7.append(num)

In [0]:
#separate x-data into classes
rcv1 = []
rcv2 = []
rcv3 = []
rcv4 = []
rcv5 = []
rcv6 = []
rcv7 = []
for num in cv1:
  rcv1.append(cov_data.data[num])
for num in cv2:
  rcv2.append(cov_data.data[num])
for num in cv3:
  rcv3.append(cov_data.data[num])
for num in cv4:
  rcv4.append(cov_data.data[num])
for num in cv5:
  rcv5.append(cov_data.data[num])
for num in cv6:
  rcv6.append(cov_data.data[num])
for num in cv7:
  rcv7.append(cov_data.data[num])

In [0]:
#even out classes
from sklearn.utils import resample
ncv1 = resample(rcv1,
               replace=False,
               n_samples=2747,
               random_state=42)
ncv2 = resample(rcv2,
               replace=False,
               n_samples=2747,
               random_state=42)
ncv3 = resample(rcv3,
               replace=False,
               n_samples=2747,
               random_state=42)
ncv4 = resample(rcv4,
               replace=False,
               n_samples=2747,
               random_state=42)
ncv5 = resample(rcv5,
               replace=False,
               n_samples=2747,
               random_state=42)
ncv6 = resample(rcv6,
               replace=False,
               n_samples=2747,
               random_state=42)
ncv7 = resample(rcv7,
               replace=False,
               n_samples=2747,
               random_state=42)

In [0]:
#create master datasets
import numpy as np
master = np.concatenate((ncv1,ncv2,ncv3,ncv4,ncv5,ncv6,ncv7))
mastert = np.concatenate((2747*[1], 2747*[2], 2747*[3], 2747*[4], 2747*[5], 2747*[6], 2747*[7]))

In [0]:
#create training and test data
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X, y = master, mastert
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

import numpy as np
newy_train = []
for i in y_train:
  f = []
  f.append(i-1)
  newy_train.append(f)
  
newy_test = []
for i in y_test:
  f = []
  f.append(i-1)
  newy_test.append(f)

newy_train = np.array(newy_train)
newy_test = np.array(newy_test)

from keras.utils.np_utils import to_categorical
print(newy_test)
y_cat_test = to_categorical(newy_test,7)
y_cat_train = to_categorical(newy_train,7)
print(y_cat_test)

In [0]:
#create first model
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 54))

classifier.add(Dropout(0.5))

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))


# Adding the output layer
classifier.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train_scaled, y_cat_train, batch_size = 10, epochs = 10)

In [0]:
#scale data and make predictions, get accuracy
X_test_scaled = scaler.transform(X_test)
y_pred = classifier.predict(X_test_scaled)
rounded_predictions = y_pred.copy()
rounded_real = []
for item in rounded_predictions:
    maxx = max(item)
    rounded_real.append(np.where(item==maxx)[0][0])

from sklearn.metrics import accuracy_score

print(accuracy_score(newy_test, rounded_real))

In [0]:
#define incorrect data function
def generate_error_imgs2(predictions, orig_imgs, real_values):
  index = -1
  count = 0
  wrong_img_indexes = []
  wrong_img_list = []
  wrong_img_class_list = []
  for item in predictions:
    index+=1
    if item != real_values[index]:
      count +=1
      wrong_img_indexes.append(index)
  print(count)
  for index2 in wrong_img_indexes:
    wrong_img_list.append(orig_imgs[index2])
    wrong_img_class_list.append(real_values[index2])
      
  return wrong_img_indexes, wrong_img_list,wrong_img_class_list

In [0]:
#get accuracy on training data
y_pred = classifier.predict(X_train_scaled)
rounded_predictions = y_pred.copy()
rounded_real = []
for item in rounded_predictions:
    maxx = max(item)
    rounded_real.append(np.where(item==maxx)[0][0])

from sklearn.metrics import accuracy_score

print(accuracy_score(newy_train, rounded_real))

In [0]:
#generate incorrect data
import numpy as np
wrong_index, wrong_x, wrong_y = generate_error_imgs2(rounded_real,X_train_scaled, newy_train)
wrong_x = np.array(wrong_x)
wrong_y = np.array(wrong_y)

In [0]:
#convert data to categorical data
from keras.utils.np_utils import to_categorical
print(wrong_y)
y_cat_wrong = to_categorical(wrong_y,7)

In [0]:
#create second model
# Initialising the ANN
classifier2 = Sequential()

# Adding the input layer and the first hidden layer
classifier2.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 54))

classifier2.add(Dropout(0.5))

# Adding the input layer and the first hidden layer
classifier2.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))


# Adding the output layer
classifier2.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier2.fit(wrong_x, y_cat_wrong, batch_size = 10, epochs = 10)

In [0]:
#fine-tune model
classifier2.fit(X_train_scaled, y_cat_train, batch_size = 10, epochs = 5)

In [0]:
#get accuracy
y_pred = classifier2.predict(X_test_scaled)
rounded_predictions = y_pred.copy()
rounded_real = []
for item in rounded_predictions:
    maxx = max(item)
    rounded_real.append((np.where(item==maxx)[0][0]))

from sklearn.metrics import accuracy_score

print(accuracy_score(newy_test, rounded_real))

In [0]:
#get accuracy on training data
y_pred = classifier2.predict(wrong_x)
rounded_predictions = y_pred.copy()
rounded_real = []
for item in rounded_predictions:
    maxx = max(item)
    rounded_real.append((np.where(item==maxx)[0][0]))

from sklearn.metrics import accuracy_score

print(accuracy_score(wrong_y, rounded_real))

In [0]:
#generate incorrect data
wrong_index2, wrong_x2, wrong_y2 = generate_error_imgs2(rounded_real,wrong_x, wrong_y)
wrong_x2 = np.array(wrong_x2)
wrong_y2 = np.array(wrong_y2)

In [0]:
#convert to categorical
from keras.utils.np_utils import to_categorical
print(wrong_y2)
y_cat_wrong2 = to_categorical(wrong_y2,7)

[[2]
 [6]
 [1]
 ...
 [2]
 [4]
 [4]]


In [0]:
#create third model
# Initialising the ANN
classifier3 = Sequential()

# Adding the input layer and the first hidden layer
classifier3.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 54))

classifier3.add(Dropout(0.5))

# Adding the input layer and the first hidden layer
classifier3.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))


# Adding the output layer
classifier3.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier3.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier3.fit(wrong_x2, y_cat_wrong2, batch_size = 10, epochs = 10)

In [0]:
#fine-tune third model on original data
classifier3.fit(X_train_scaled, y_cat_train, batch_size = 10, epochs = 5)

In [0]:
#get accuracy on test data
y_pred = classifier3.predict(X_test_scaled)
rounded_predictions = y_pred.copy()
rounded_real = []
for item in rounded_predictions:
    maxx = max(item)
    rounded_real.append((np.where(item==maxx)[0][0]))

from sklearn.metrics import accuracy_score

print(accuracy_score(newy_test, rounded_real))

In [0]:
#get accuracy on training data
y_pred = classifier3.predict(wrong_x2)
rounded_predictions = y_pred.copy()
rounded_real = []
for item in rounded_predictions:
    maxx = max(item)
    rounded_real.append((np.where(item==maxx)[0][0]))

from sklearn.metrics import accuracy_score

print(accuracy_score(wrong_y2, rounded_real))

In [0]:
#generate incorrect data
wrong_index3, wrong_x3, wrong_y3 = generate_error_imgs2(rounded_real,wrong_x2, wrong_y2)
wrong_x3 = np.array(wrong_x3)
wrong_y3 = np.array(wrong_y3)

In [0]:
#convert to categorical
from keras.utils.np_utils import to_categorical
print(wrong_y3)
y_cat_wrong3 = to_categorical(wrong_y3,7)

In [0]:
#fine-tune first model on incorrect data
classifier3.fit(wrong_x3, y_cat_wrong3, batch_size = 10, epochs = 5)

In [0]:
#get final predictions
firstpred = classifier.predict(X_train_scaled)
secondpred = classifier2.predict(X_train_scaled)
thirdpred = classifier3.predict(X_train_scaled)

In [0]:
#create master data
master = []
for i in range(len(X_train_scaled)):
  one = [None] * 21
  one[0] = firstpred[i][0]
  one[1] = firstpred[i][1]
  one[2] = firstpred[i][2]
  one[3] = firstpred[i][3]
  one[4] = firstpred[i][4]
  one[5] = firstpred[i][5]
  one[6] = firstpred[i][6]
  one[7] = secondpred[i][0]
  one[8] = secondpred[i][1]
  one[9] = secondpred[i][2]
  one[10] = secondpred[i][3]
  one[11] = secondpred[i][4]
  one[12] = secondpred[i][5]
  one[13] = secondpred[i][6]
  one[14] = thirdpred[i][0]
  one[15] = thirdpred[i][1]
  one[16] = thirdpred[i][2]
  one[17] = thirdpred[i][3]
  one[18] = thirdpred[i][4]
  one[19] = thirdpred[i][5]
  one[20] = thirdpred[i][6]
  master.append(one)
master = np.array(master)

In [0]:
#create final model
# Initialising the ANN
mclassifier = Sequential()

# Adding the input layer and the first hidden layer
mclassifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 21))

mclassifier.add(Dropout(.5))

# Adding the second hidden layer
mclassifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
mclassifier.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
mclassifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
mclassifier.fit(master, y_cat_train, batch_size = 10, epochs = 10)

In [0]:
#create master test data
firstpred = classifier.predict(X_test_scaled)
secondpred = classifier2.predict(X_test_scaled)
thirdpred = classifier3.predict(X_test_scaled)
master2 = []
for i in range(len(X_test)):
  one = [None] * 21
  one[0] = firstpred[i][0]
  one[1] = firstpred[i][1]
  one[2] = firstpred[i][2]
  one[3] = firstpred[i][3]
  one[4] = firstpred[i][4]
  one[5] = firstpred[i][5]
  one[6] = firstpred[i][6]
  one[7] = secondpred[i][0]
  one[8] = secondpred[i][1]
  one[9] = secondpred[i][2]
  one[10] = secondpred[i][3]
  one[11] = secondpred[i][4]
  one[12] = secondpred[i][5]
  one[13] = secondpred[i][6]
  one[14] = thirdpred[i][0]
  one[15] = thirdpred[i][1]
  one[16] = thirdpred[i][2]
  one[17] = thirdpred[i][3]
  one[18] = thirdpred[i][4]
  one[19] = thirdpred[i][5]
  one[20] = thirdpred[i][6]
  master2.append(one)
master2 = np.array(master2)

In [0]:
#get final accuracy
y_pred = mclassifier.predict(master2)
rounded_predictions = y_pred.copy()
rounded_real = []
for item in rounded_predictions:
    maxx = max(item)
    rounded_real.append((np.where(item==maxx)[0][0]))

from sklearn.metrics import confusion_matrix,classification_report
report = classification_report(newy_test, rounded_real)
print(report) 
print(confusion_matrix(newy_test,rounded_real))